<a href="https://colab.research.google.com/github/TonyQ2k3/pytorch-deeplearning/blob/main/fundamentals/introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Pytorch

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

## Working with Data
PyTorch has 2 primitives to work with data: `torch.utils.data.Dataset` and `torch.utils.data.DataLoader`.

`Dataset` contains data and labels, `DataLoader` turns it into an iterable so we can work with them.

In [2]:
training_data = datasets.FashionMNIST(
  root='data',
  train=True,
  transform=ToTensor(),
  download=True
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    transform=ToTensor(),
    download=True
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 18.6MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 309kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 5.50MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 13.1MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [4]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )
  def forward(self, x):
    x_flattened = self.flatten(x)
    logits = self.stack(x_flattened)
    return logits

model = NeuralNetwork()

In [5]:
# Hyperparams
## Epochs: Number of times the dataset is passed through the model
## Batch size: Number of samples the model sees each epochs
## Step size (learning rate): Steps the model take to find a better weight. Too small => Take long time; Too large => May skip the best weight

learning_rate = 0.001
epochs = 5
batch_size = 64

# Loss func
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
# Training
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    # Calculate loss
    y_preds = model(X)
    loss = loss_fn(y_preds, y)

    # Optimizing
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# Testing
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      y_preds = model(X)
      test_loss += loss_fn(y_preds, y).item()
      correct += (y_preds.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= size
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
# Start training

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301724  [    0/60000]
loss: 2.299560  [ 6400/60000]
loss: 2.286786  [12800/60000]
loss: 2.282403  [19200/60000]
loss: 2.273354  [25600/60000]
loss: 2.262784  [32000/60000]
loss: 2.275910  [38400/60000]
loss: 2.250309  [44800/60000]
loss: 2.244017  [51200/60000]
loss: 2.240890  [57600/60000]
Test Error: 
 Accuracy: 40.3%, Avg loss: 0.034972 

Epoch 2
-------------------------------
loss: 2.242568  [    0/60000]
loss: 2.230204  [ 6400/60000]
loss: 2.185237  [12800/60000]
loss: 2.190307  [19200/60000]
loss: 2.175929  [25600/60000]
loss: 2.159860  [32000/60000]
loss: 2.202749  [38400/60000]
loss: 2.151306  [44800/60000]
loss: 2.151529  [51200/60000]
loss: 2.142649  [57600/60000]
Test Error: 
 Accuracy: 40.2%, Avg loss: 0.033171 

Epoch 3
-------------------------------
loss: 2.154103  [    0/60000]
loss: 2.120513  [ 6400/60000]
loss: 2.025701  [12800/60000]
loss: 2.047816  [19200/60000]
loss: 2.023637  [25600/60000]
loss: 2.006219  [32000/600

In [8]:
torch.save(model.state_dict(), "data/model.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# Loading and running model

Setting the model to evaluation mode is important
+ **Dropout layers** disable some activation neurons to prevent overfitting. Setting this to eval will allows all neurons to be used.
+ **Batch normalization** layer computes the mean and variance of batch data during training, setting this to eval will allow these statistics to be used.

In [9]:
new_model = NeuralNetwork()
new_model.load_state_dict(torch.load('data/model.pth'))

# Calling eval() before inferencing to set the dropout & batch normalization to evaluation mode
new_model.eval()

<ipython-input-9-e037dbd45f0f>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  new_model.load_state_dict(torch.load('data/model.pth'))


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)